#Interactive_N_N

This code implements a modular feedforward neural network with the following features:

1)Layer Class:

Represents individual layers with customizable activation functions (ReLU, Sigmoid, Tanh).
Handles forward propagation and activation operations.

2)NeuralNetwork Class:

Manages the network architecture with multiple layers.

Supports:

Forward Propagation: Computes predictions.
Backward Propagation: Updates weights using gradient descent.
Training: Mini-batch training with loss and accuracy tracking.
Prediction: Outputs class probabilities or predictions.
Tracks training history and plots loss and accuracy.

3)Save/Load Functionality:

Save model architecture, weights, and biases to a JSON file.
Load and reconstruct models from saved files.

4)Visualization:

Plots loss and accuracy trends during training.

In [1]:
import numpy as np
from typing import List, Tuple, Optional
import json
import matplotlib.pyplot as plt

class Layer:
    def __init__(self, input_size: int, output_size: int, activation: str = 'relu'):
        """Initialize a neural network layer with weights and biases"""
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros((1, output_size))
        self.activation = activation

        # Cache for backpropagation
        self.input = None
        self.output = None
        self.activation_output = None

    def forward(self, X: np.ndarray) -> np.ndarray:
        """Forward pass through the layer"""
        self.input = X
        self.output = np.dot(X, self.weights) + self.biases
        self.activation_output = self._activate(self.output)
        return self.activation_output

    def _activate(self, x: np.ndarray) -> np.ndarray:
        """Apply activation function"""
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'tanh':
            return np.tanh(x)
        raise ValueError(f"Unsupported activation function: {self.activation}")

    def _activate_derivative(self, x: np.ndarray) -> np.ndarray:
        """Compute derivative of activation function"""
        if self.activation == 'relu':
            return np.where(x > 0, 1, 0)
        elif self.activation == 'sigmoid':
            s = 1 / (1 + np.exp(-x))
            return s * (1 - s)
        elif self.activation == 'tanh':
            return 1 - np.tanh(x)**2
        raise ValueError(f"Unsupported activation function: {self.activation}")



class NeuralNetwork:
    def __init__(self, layer_sizes: List[int], activations: List[str]):
        """Initialize neural network with specified layer sizes and activations"""
        if len(layer_sizes) < 2:
            raise ValueError("Need at least input and output layers")
        if len(layer_sizes) - 1 != len(activations):
            raise ValueError("Number of activations must match number of layers - 1")

        self.layers = []
        for i in range(len(layer_sizes) - 1):
            self.layers.append(Layer(layer_sizes[i], layer_sizes[i + 1], activations[i]))

        self.loss_history = []
        self.accuracy_history = []

    def forward(self, X: np.ndarray) -> np.ndarray:
        """Forward pass through entire network"""
        current_output = X
        for layer in self.layers:
            current_output = layer.forward(current_output)
        return current_output

    def backward(self, X: np.ndarray, y: np.ndarray, learning_rate: float = 0.01):
        """Backward pass for parameter updates"""
        m = X.shape[0]

        # Compute initial gradient
        output = self.layers[-1].activation_output
        delta = output - y

        # Backpropagate through layers
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]

            # Compute gradients
            if i != len(self.layers) - 1:  # Hidden layers
                delta = np.dot(delta, self.layers[i + 1].weights.T) * layer._activate_derivative(layer.output)

            # Update parameters
            layer.weights -= learning_rate * np.dot(layer.input.T, delta) / m
            layer.biases -= learning_rate * np.sum(delta, axis=0, keepdims=True) / m

    def train(self, X: np.ndarray, y: np.ndarray, epochs: int = 1000, learning_rate: float = 0.01,
             batch_size: Optional[int] = None, verbose: bool = True) -> None:
        """Train the neural network"""
        m = X.shape[0]
        batch_size = batch_size if batch_size else m

        for epoch in range(epochs):
            # Mini-batch training
            for i in range(0, m, batch_size):
                batch_X = X[i:i + batch_size]
                batch_y = y[i:i + batch_size]

                # Forward and backward passes
                self.forward(batch_X)
                self.backward(batch_X, batch_y, learning_rate)

            # Track metrics
            predictions = self.predict(X)
            loss = self.compute_loss(predictions, y)
            accuracy = self.compute_accuracy(predictions, y)

            self.loss_history.append(loss)
            self.accuracy_history.append(accuracy)

            if verbose and epoch % 100 == 0:
                print(f"Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Make predictions"""
        return self.forward(X)

    def compute_loss(self, predictions: np.ndarray, y: np.ndarray) -> float:
        """Compute MSE loss"""
        return np.mean(np.square(predictions - y))

    def compute_accuracy(self, predictions: np.ndarray, y: np.ndarray) -> float:
        """Compute classification accuracy"""
        return np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))

    def plot_training_history(self) -> None:
        """Plot training metrics history"""
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(self.loss_history)
        plt.title('Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.subplot(1, 2, 2)
        plt.plot(self.accuracy_history)
        plt.title('Training Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.tight_layout()
        plt.show()

    def save_model(self, filepath: str) -> None:
        """Save model parameters to file"""
        model_params = {
            'architecture': [layer.weights.shape[0] for layer in self.layers] +
                          [self.layers[-1].weights.shape[1]],
            'activations': [layer.activation for layer in self.layers],
            'weights': [layer.weights.tolist() for layer in self.layers],
            'biases': [layer.biases.tolist() for layer in self.layers]
        }
        with open(filepath, 'w') as f:
            json.dump(model_params, f)

    @classmethod
    def load_model(cls, filepath: str) -> 'NeuralNetwork':
        """Load model from file"""
        with open(filepath, 'r') as f:
            model_params = json.load(f)

        network = cls(model_params['architecture'], model_params['activations'])
        for i, layer in enumerate(network.layers):
            layer.weights = np.array(model_params['weights'][i])
            layer.biases = np.array(model_params['biases'][i])

        return network